In [1]:
import coax
import gym
import haiku as hk
import jax
import jax.numpy as jnp
import optax
from coax.value_losses import mse



# the name of this script
name = 'a2c'

# the cart-pole MDP
env = gym.make('CartPole-v0')
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")

In [2]:
# ## Baseline MLPS

# def func_pi(S, is_training):
#     logits = hk.Sequential((
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(env.action_space.n, w_init=jnp.zeros)
#     ))
#     return {'logits': logits(S)}


# def func_v(S, is_training):
#     value = hk.Sequential((
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(8), jax.nn.relu,
#         hk.Linear(1, w_init=jnp.zeros), jnp.ravel
#     ))
#     return value(S)



In [3]:
from emlp import T, Scalar
from emlp.groups import SO, S, O, Trivial,Z
from emlp_haiku import EMLPBlock, Sequential, Linear,EMLP
from emlp.reps import Rep
from emlp.nn import gated,gate_indices,uniform_rep
from math import prod
from representations import PseudoScalar

## Trivial
# group=Trivial(2)
# rep_in = T(0)*prod(env.observation_space.shape)
# rep_out = T(0)*env.action_space.n#prod(env.action_space.shape)

## Reflection
group=Z(2)
rep_in = PseudoScalar()*prod(env.observation_space.shape)
rep_out = PseudoScalar()*env.action_space.n#prod(env.action_space.shape)

nn_pi = EMLP(rep_in,rep_out,group,ch=100,num_layers=2)
nn_v = EMLP(rep_in,T(0),group,ch=100,num_layers=2)

def func_pi(S, is_training):
    return {'logits': nn_pi(S)}


def func_v(S, is_training):
    return nn_v(S).reshape(-1)

[a2c|root|INFO] Initing EMLP
[a2c|root|INFO] Linear W components:400 rep:96P+48P⊗V+20P⊗V²+8P⊗V³+4P⊗V⁴
[a2c|root|INFO] P cache miss
[a2c|root|INFO] Solving basis for P, for G=Z(2)


(P+P+P+P) (P+P) 4P 2P
[4P, 24V⁰+12V+5V²+2V³+V⁴, 24V⁰+12V+5V²+2V³+V⁴]


[a2c|root|INFO] P⊗V cache miss
[a2c|root|INFO] Solving basis for P⊗V, for G=Z(2)
[a2c|root|INFO] P⊗V² cache miss
[a2c|root|INFO] Solving basis for P⊗V², for G=Z(2)
[a2c|root|INFO] P⊗V³ cache miss
[a2c|root|INFO] Solving basis for P⊗V³, for G=Z(2)
[a2c|root|INFO] P⊗V⁴ cache miss
[a2c|root|INFO] Solving basis for P⊗V⁴, for G=Z(2)
[a2c|root|INFO] V cache miss
[a2c|root|INFO] Solving basis for V, for G=Z(2)
[a2c|root|INFO] V² cache miss
[a2c|root|INFO] Solving basis for V², for G=Z(2)
[a2c|root|INFO] V³ cache miss
[a2c|root|INFO] Solving basis for V³, for G=Z(2)
[a2c|root|INFO] V⁴ cache miss
[a2c|root|INFO] Solving basis for V⁴, for G=Z(2)
[a2c|root|INFO] Linear W components:10000 rep:576V⁰+576V+384V²+216V³+121V⁴+44V⁵+14V⁶+4V⁷+V⁸
[a2c|root|INFO] V⁵ cache miss
[a2c|root|INFO] Solving basis for V⁵, for G=Z(2)
[a2c|root|INFO] V⁶ cache miss
[a2c|root|INFO] Solving basis for V⁶, for G=Z(2)
[a2c|root|INFO] V⁷ cache miss
[a2c|root|INFO] Solving basis for V⁷, for G=Z(2)
[a2c|root|INFO] V⁸ cache mi

[4P, 24V⁰+12V+5V²+2V³+V⁴, 24V⁰+12V+5V²+2V³+V⁴]


[a2c|root|INFO] Linear W components:100 rep:24V⁰+12V+5V²+2V³+V⁴


In [4]:

# these optimizers collect batches of grads before applying updates
optimizer_v = optax.chain(optax.apply_every(k=32), optax.adam(0.002))
optimizer_pi = optax.chain(optax.apply_every(k=32), optax.adam(0.001))


# value function and its derived policy
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)

# experience tracer
tracer = coax.reward_tracing.NStep(n=1, gamma=0.9)

# updaters
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optimizer_pi)
simple_td = coax.td_learning.SimpleTD(v, loss_function=mse, optimizer=optimizer_v)


# train
for ep in range(1000):
    s = env.reset()
    er = 0
    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)
        
        if done and (t == env.spec.max_episode_steps - 1):
            r = 1 / (1 - tracer.gamma)
        er+=r
        tracer.add(s, a, r, done)
        while tracer:
            transition_batch = tracer.pop()
            metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
            metrics_pi = vanilla_pg.update(transition_batch, td_error)
            env.record_metrics(metrics_v)
            env.record_metrics(metrics_pi)

        if done:
            break

        s = s_next
    
    print("Epoch reward",er)
    # early stopping
    if env.avg_G > env.spec.reward_threshold:
        break


# run env one more time to render
#coax.utils.generate_gif(env, policy=pi, filepath=f"./data/{name}.gif", duration=25)

[a2c|TrainMonitor|INFO] ep: 1,	T: 28,	G: 27,	avg_r: 1,	avg_G: 27,	t: 27,	dt: 309.307ms,	SimpleTD/loss: 0.469,	VanillaPG/loss: 0.665
[a2c|TrainMonitor|INFO] ep: 2,	T: 40,	G: 11,	avg_r: 1,	avg_G: 19,	t: 11,	dt: 13.902ms,	SimpleTD/loss: 0.419,	VanillaPG/loss: 0.616


Epoch reward 27.0
Epoch reward 11.0


[a2c|TrainMonitor|INFO] ep: 3,	T: 56,	G: 15,	avg_r: 1,	avg_G: 17.7,	t: 15,	dt: 14.357ms,	SimpleTD/loss: 0.366,	VanillaPG/loss: 0.532
[a2c|TrainMonitor|INFO] ep: 4,	T: 69,	G: 12,	avg_r: 1,	avg_G: 16.2,	t: 12,	dt: 13.843ms,	SimpleTD/loss: 0.356,	VanillaPG/loss: 0.55


Epoch reward 15.0
Epoch reward 12.0


[a2c|TrainMonitor|INFO] ep: 5,	T: 91,	G: 21,	avg_r: 1,	avg_G: 17.2,	t: 21,	dt: 14.643ms,	SimpleTD/loss: 0.369,	VanillaPG/loss: 0.406


...

Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 88,	T: 10,414,	G: 200,	avg_r: 1,	avg_G: 194,	t: 200,	dt: 13.609ms,	SimpleTD/loss: 4.3e-05,	VanillaPG/loss: -0.000171


Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 89,	T: 10,615,	G: 200,	avg_r: 1,	avg_G: 195,	t: 200,	dt: 13.985ms,	SimpleTD/loss: 1.88e-05,	VanillaPG/loss: -0.000526


Epoch reward 209.0
Epoch reward 209.0
